<a href="https://colab.research.google.com/github/MichaelGelo/GRP2_CEPARCO_IP/blob/main/CEPARCO_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group 2 - Implementing Hyyrö’s Bit Vector Algorithm Using CUDA SIMT**
## **GROUP 2 - S11**

**MEMBERS:**

- Alfred Bastin S. Agustines
- Allan David C. De Leon
- Michael Angelo Depasucat
- Kai Hiori J. Padilla


##Check if CUDA is present

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr  2 03:30:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename, content in uploaded.items():
    with open(filename, "wb") as f:
        f.write(content)

KeyboardInterrupt: 

In [2]:
%%writefile C_utils.h
#ifdef __cplusplus
extern "C" {
#endif

char* read_file_into_string(const char* filename);
char** parse_fasta_file(const char *filename, int *num_sequences);

#ifdef __cplusplus
}
#endif

Writing C_utils.h


In [3]:
%%writefile C_utils.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "C_utils.h"

#define MAX_LENGTH (1 << 24)
#define MAX_LINE_LENGTH (1 << 14)

char* read_file_into_string(const char* filename) {
    FILE* file = fopen(filename, "rb");
    if (!file) {
        perror("Failed to open file");
        return NULL;
    }
    fseek(file, 0, SEEK_END);
    long file_size = ftell(file);
    rewind(file);
    char* buffer = (char*)malloc(file_size + 1);
    if (!buffer) {
        perror("Failed to allocate memory");
        fclose(file);
        return NULL;
    }
    size_t bytes_read = fread(buffer, 1, file_size, file);
    if (bytes_read != file_size) {
        perror("Failed to read the file completely");
        free(buffer);
        fclose(file);
        return NULL;
    }
    buffer[file_size] = '\0';
    fclose(file);
    return buffer;
}

char** parse_fasta_file(const char *filename, int *num_sequences) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        perror("Failed to open FASTA file");
        return NULL;
    }
    char **sequences = NULL;
    int seq_count = 0;
    char *current_seq = NULL;
    size_t current_seq_len = 0;
    char line[MAX_LINE_LENGTH];

    while (fgets(line, sizeof(line), file)) {
        if (line[0] == '>') {
            if (current_seq != NULL) {
                if (current_seq_len > 0) {
                    sequences = (char**)realloc(sequences, (seq_count + 1) * sizeof(char*));
                    sequences[seq_count] = (char*)malloc(current_seq_len + 1);
                    memcpy(sequences[seq_count], current_seq, current_seq_len);
                    sequences[seq_count][current_seq_len] = '\0';
                    seq_count++;
                }
                free(current_seq);
                current_seq = NULL;
                current_seq_len = 0;
            }
        } else {
            size_t line_len = strlen(line);
            while (line_len > 0 && (line[line_len - 1] == '\n' || line[line_len - 1] == '\r')) {
                line_len--;
            }
            if (current_seq_len + line_len > MAX_LENGTH) {
                line_len = MAX_LENGTH - current_seq_len;
            }
            if (line_len > 0) {
                current_seq = (char*)realloc(current_seq, current_seq_len + line_len + 1);
                memcpy(current_seq + current_seq_len, line, line_len);
                current_seq_len += line_len;
                current_seq[current_seq_len] = '\0';
            }
        }
    }
    if (current_seq != NULL && current_seq_len > 0) {
        sequences = (char**)realloc(sequences, (seq_count + 1) * sizeof(char*));
        sequences[seq_count] = (char*)malloc(current_seq_len + 1);
        memcpy(sequences[seq_count], current_seq, current_seq_len);
        sequences[seq_count][current_seq_len] = '\0';
        seq_count++;
        free(current_seq);
    }
    fclose(file);
    *num_sequences = seq_count;
    return sequences;
}


Writing C_utils.c


##C - Single Query & Multiple Reference

In [ ]:
%%writefile C_hyyro.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <time.h>
#include "C_utils.h"

#define MAX_LENGTH (1 << 24)

#define query_file "Query_1.txt"
#define reference_file "Reference_87.fasta"
#define loope 10
typedef uint64_t bitvector;

int bit_vector_levenshtein(const char *query, const char *reference, int *zero_indices, int *zero_count, int* lowest) {
    int m = strlen(query);
    int n = strlen(reference);
    if (m > MAX_LENGTH || n > MAX_LENGTH) {
        printf("Error: Strings too long for this implementation!\n");
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Eq[256] = {0};
    bitvector Ph, Mh, Xv, Xh, Xp;

    for(int i = 0; i < m; i++) {
        Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    int score = m;
    *zero_count = 0;
	  *lowest = m;

    for(int j = 0; j < n; j++) {

        Xv = Eq[(unsigned char)reference[j]] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((Xv & Pv) + Pv) ^ Pv | Xv | Mv;
        Ph = Mv | ~(Xh | Pv);
        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (m - 1))) score++;
        if (Mh & (1ULL << (m - 1))) score--;

        if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }

    return score;
}

int main() {
    clock_t start, end;
    double elapsed_total = 0.0;

    // Read & Print Query
    const char *query = read_file_into_string(query_file);
    printf("Query Sequence:\n%s\n\n", query);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);

    // Print Reference
    printf("Reference Sequences:\n");
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }

    // Process each ref
    for (int i = 0; i < num_references; i++) {
        int distance = 0;
        int zero_count = 0;
        int lowest_score = 0;
        // Alloc
        int *zero_indices = (int*)malloc(strlen(reference_seqs[i]) * sizeof(int));
        if (!zero_indices) {
            perror("Failed to allocate memory for zero_indices");
            continue;
        }
        // Time
        for (int j = 0; j < loope; j++) {
            start = clock();
            distance = bit_vector_levenshtein(query, reference_seqs[i], zero_indices, &zero_count, &lowest_score);
            end = clock();
            elapsed_total += ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        }

        // print result lng
        printf("Results for Reference #%d:\n", i + 1);
        printf("Distance: %d, Lowest Score: %d, Hits (zero score positions): %d\n",
               distance, lowest_score, zero_count);
        printf("Zero Indices: ");
        for (int k = 0; k < zero_count; k++) {
            printf("%d ", zero_indices[k]);
        }
        printf("\n\n");

        free(zero_indices);
    }

    printf("Function (C - Single Query & Multiple Reference) average time for %u loops is %f milliseconds\n", loope, elapsed_total / loope);

    // Free mem
    free((void*)query);
    for (int i = 0; i < num_references; i++) {
        free(reference_seqs[i]);
    }
    free(reference_seqs);

    return 0;
}


Overwriting C_hyyro.c


In [ ]:
%%shell
gcc C_hyyro.c C_utils.c -o C_hyyro

In [ ]:
%%shell
./C_hyyro

Query Sequence:
TCC

Reference Sequences:
Reference #1:
TAGCAACGGTAGCAGTCGTAAAGGTGGTAGTAGAATTTATATTAGTACTATCAGCACATCAAAATAGTGTTTGCTTTGGAGAGAAGAAATCACGGAATACCGGCCAATCCCTCAACTACGAGATACCCCGTCTGAGCGCACAGGCATTGTTCTGTGGCGCGTAGGAATGCCCTTCCCTGCATCCGCTAGCGATAGATTGCATGCTCAGCTGCCGTCTGTGCGGGATCGCTTGGCCTCACGCTCAGCCTCGCCTCCCGTCGGCGGGCGCCTGACTGCAGCCACTGCTCGCAGTGTGTCAGCCCGCGTGAACGCGTGCATCAGCACAGCGCGTGCGTTGGTGCCGAGAGCGCGCTTTGACAGATCGCCATTGAGCTGTAGTGCCTCGCCGAGATTAGCAATGTCGTAGGGCGTCCGCAAGTCCAATGCCTGGTGCGCGCCGCTTGTCCACACTAGTCCACGGCCCCGTGTCGCGCGCACAACGCCTGCTGCATTGCTAACCCATTGCTGTCGCGTGGTCGCGTCTACTAGCGCGGGGCGATATGCGATCTCTAGCGATAGCCCGTTTGCTAGCGCCTGGCCAACCATCTTGTGCTTTGCAAAAAATCCCCACCGTGCCCCCATATCCAGCGTTACAAGATCAACCGCATCCCAGGCGCTGCTGCACGCTGCAAGAAGTATCTTTTCTGACACTGGCTGCACCGCGACAACGTCGTACTCGCTCGCTGTTGGGCTGCTGTTGCTGAGGCTATGCCCATGCGCTGCATCTGAAATCACCGCTGTCACCCTGCGCAGTATGCGGATCTTGCCTCGTGGAACTCGTCCCGTTGCCCCCTCGCTTGTCCGCGTGCCTGTCTTGCGCTCCCAGCTCAGCACTGCATCTTTGAACGCAGGTGCTGACTTCCACACAGCTAGATGCTCCGGCACCAATCGCCCCTGCACTGTCT

##C - Multiple Query & Multiple Reference

In [ ]:
%%writefile C_hyyro_multiple.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <time.h>
#include "C_utils.h"

#define MAX_LENGTH (1 << 24)

#define query_file "Query_3.fasta"
#define reference_file "Reference_87.fasta"
#define loope 10

typedef uint64_t bitvector;

int bit_vector_levenshtein(const char *query, const char *reference, int *zero_indices, int *zero_count, int* lowest) {
    int m = strlen(query);
    int n = strlen(reference);
    if (m > MAX_LENGTH || n > MAX_LENGTH) {
        printf("Error: Strings too long for this implementation!\n");
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Eq[256] = {0};
    bitvector Ph, Mh, Xv, Xh, Xp;

    for(int i = 0; i < m; i++) {
        Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    int score = m;
    *zero_count = 0;
	  *lowest = m;

    for(int j = 0; j < n; j++) {

        Xv = Eq[(unsigned char)reference[j]] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((Xv & Pv) + Pv) ^ Pv | Xv | Mv;
        Ph = Mv | ~(Xh | Pv);

        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (m - 1))) score++;
        if (Mh & (1ULL << (m - 1))) score--;
		    if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;


        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }
    return score;
}

int main() {
    clock_t start, end;
    double elapsed_total = 0.0;

    int num_queries = 0, num_references = 0;
    // Read & Print Query
    char **query_seqs = parse_fasta_file(query_file, &num_queries);
    for (int i = 0; i < num_queries; i++) {
        printf("Query #%d:\n%s\n\n", i + 1, query_seqs[i]);
    }

    // Read Reference
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }

    // Process each ref for each query
    for (int q = 0; q < num_queries; q++) {
        for (int r = 0; r < num_references; r++) {
            int distance = 0, zero_count = 0, lowest_score = 0;
            int *zero_indices = (int*)malloc(strlen(reference_seqs[r]) * sizeof(int));
            if (!zero_indices) {
                perror("Failed to allocate memory for zero_indices");
                continue;
            }
            // Time with loope
            for (int j = 0; j < loope; j++) {
            start = clock();
            distance = bit_vector_levenshtein(query_seqs[q], reference_seqs[r], zero_indices, &zero_count, &lowest_score);
            end = clock();
            elapsed_total += ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
            }

            printf("Query #%d vs Reference #%d:\n", q + 1, r + 1);
            printf("Distance: %d, Lowest Score: %d, Hits (zero score positions): %d\n", distance, lowest_score, zero_count);
            printf("Zero Indices: ");
            for (int k = 0; k < zero_count; k++) {
                printf("%d ", zero_indices[k]);
            }
            printf("\n\n");

            free(zero_indices);
        }
    }

    printf("Function (C - Multiple Query & Multiple Reference) average time for %u loops is %f milliseconds\n", loope, elapsed_total / loope);

    // Free mem
    for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
    free(query_seqs);
    for (int i = 0; i < num_references; i++) free(reference_seqs[i]);
    free(reference_seqs);

    return 0;
}


Overwriting C_hyyro_multiple.c


In [ ]:
%%shell
gcc C_hyyro_multiple.c -o C_hyyro_multiple C_utils.c

In [ ]:
%%shell
./C_hyyro_multiple

Query #1:
TAG

Query #2:
TGGGTT

Query #3:
AGCGTT

Reference #1:
TAGCAACGGTAGCAGTCGTAAAGGTGGTAGTAGAATTTATATTAGTACTATCAGCACATCAAAATAGTGTTTGCTTTGGAGAGAAGAAATCACGGAATACCGGCCAATCCCTCAACTACGAGATACCCCGTCTGAGCGCACAGGCATTGTTCTGTGGCGCGTAGGAATGCCCTTCCCTGCATCCGCTAGCGATAGATTGCATGCTCAGCTGCCGTCTGTGCGGGATCGCTTGGCCTCACGCTCAGCCTCGCCTCCCGTCGGCGGGCGCCTGACTGCAGCCACTGCTCGCAGTGTGTCAGCCCGCGTGAACGCGTGCATCAGCACAGCGCGTGCGTTGGTGCCGAGAGCGCGCTTTGACAGATCGCCATTGAGCTGTAGTGCCTCGCCGAGATTAGCAATGTCGTAGGGCGTCCGCAAGTCCAATGCCTGGTGCGCGCCGCTTGTCCACACTAGTCCACGGCCCCGTGTCGCGCGCACAACGCCTGCTGCATTGCTAACCCATTGCTGTCGCGTGGTCGCGTCTACTAGCGCGGGGCGATATGCGATCTCTAGCGATAGCCCGTTTGCTAGCGCCTGGCCAACCATCTTGTGCTTTGCAAAAAATCCCCACCGTGCCCCCATATCCAGCGTTACAAGATCAACCGCATCCCAGGCGCTGCTGCACGCTGCAAGAAGTATCTTTTCTGACACTGGCTGCACCGCGACAACGTCGTACTCGCTCGCTGTTGGGCTGCTGTTGCTGAGGCTATGCCCATGCGCTGCATCTGAAATCACCGCTGTCACCCTGCGCAGTATGCGGATCTTGCCTCGTGGAACTCGTCCCGTTGCCCCCTCGCTTGTCCGCGTGCCTGTCTTGCGCTCCCAGCTCAGCACTGCATCTTTGAACGCAGGTGCTGACTTCCACACAGCTAGATGCTCCGGCACCAATCGCCCCT

##CUDA - Single Query & Multiple Reference

In [14]:
%%writefile CUDA.cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include "C_utils.h"

// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)
#define MAX_HITS 1024

// params to change here ↓↓↓

#define query_file "Query_1.txt"
#define reference_file "Reference_87.fasta"
#define threadsPerBlock 256
#define loope 10

// params to change here ↑↑↑


typedef uint64_t bitvector;
__constant__ bitvector d_Eq[256];

__host__ __device__ int bit_vector_levenshtein(int query_length, const char *reference, int reference_length, const bitvector *Eq, int *zero_indices, int *zero_count, int* lowest) {
    if (query_length > MAX_LENGTH || reference_length > MAX_LENGTH) {
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Ph = 0;
    bitvector Mh = 0;
    bitvector Xv = 0;
    bitvector Xh = 0;
    bitvector Xp = 0;
    int score = query_length;


    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; j++) {
        unsigned char c = reference[j];

        Xv = Eq[c] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((((Xv & Pv) + Pv) ^ Pv) | Xv | Mv);
        Ph = Mv | ~(Xh | Pv);

        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (query_length - 1))) score++;
        if (Mh & (1ULL << (query_length - 1))) score--;
        if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }
    return score;
}

__global__ void levenshtein_kernel(int query_length, const char *__restrict__ references, const int *__restrict__ reference_lengths, int *__restrict__ distances,
                                   int *__restrict__ zero_counts, int *__restrict__ zero_indices, int num_references, int *__restrict__ d_lowest) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < num_references) {
        const char *reference = &references[idx * MAX_LENGTH];
        int reference_length = reference_lengths[idx];
        int *current_zero_indices = &zero_indices[idx * MAX_HITS];

        distances[idx] = bit_vector_levenshtein(
            query_length,
            reference,
            reference_length,
            d_Eq,
            current_zero_indices,
            &zero_counts[idx],
            &d_lowest[idx]
        );
    }
}

int main(){
    // Read Query
    const char* query = read_file_into_string(query_file);
    int query_length = strlen(query);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    int *reference_lengths = (int*)malloc(num_references * sizeof(int));
    if (!reference_seqs) {
        free((void*)query);
        return 1;
    }

    for (int i = 0; i < num_references; i++) {
        reference_lengths[i] = strlen(reference_seqs[i]);
    }


    // Precompute Eq for the query and store it in host memory.
    bitvector h_Eq[256] = {0};
    for (int i = 0; i < query_length; i++) {
        h_Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    // Copy Eq to constant memory on the device.
    cudaMemcpyToSymbol(d_Eq, h_Eq, sizeof(bitvector) * 256);

    // Allocate Unified Memory
    char *references;
    int *d_reference_lengths, *d_distances, *d_zero_indices, *d_zero_counts, *d_lowest;
    cudaMallocManaged(&references, num_references * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_reference_lengths, num_references * sizeof(int));
    cudaMallocManaged(&d_distances, num_references * sizeof(int));
    cudaMallocManaged(&d_zero_indices, num_references * MAX_HITS * sizeof(int));
    cudaMallocManaged(&d_zero_counts, num_references * sizeof(int));
    cudaMallocManaged(&d_lowest, num_references * sizeof(int));

    // copy ref and ref_length to managed memory
    for (int i = 0; i < num_references; i++) {
        strncpy(&references[i * MAX_LENGTH], reference_seqs[i], MAX_LENGTH);
        d_reference_lengths[i] = reference_lengths[i];
    }

    // get gpu id
    int device = -1;
    cudaGetDevice(&device);


    // memory advise
    cudaMemAdvise(references, num_references * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_reference_lengths, num_references * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_distances, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(d_zero_indices, num_references * MAX_HITS * sizeof(int), cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(d_zero_counts, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(d_lowest, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);


    // prefetch data
    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_distances, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_zero_indices, num_references * MAX_HITS * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_zero_counts, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), device, NULL);

    // setup CUDA kernel
    int blocksPerGrid = (num_references + threadsPerBlock - 1) / threadsPerBlock;

    // print infos
    printf("*** function = Levenshtein Distance\n");
    printf("numReferences = %d\n", num_references);
    printf("numBlocks = %d, numThreads = %d\n", blocksPerGrid, threadsPerBlock);

    // Execute kernel multiple times
    for (size_t i = 0; i < loope; i++) {
        levenshtein_kernel<<<blocksPerGrid, threadsPerBlock>>>(query_length, references, d_reference_lengths, d_distances, d_zero_counts, d_zero_indices, num_references, d_lowest);
    }

    // barrier
    cudaDeviceSynchronize();

    // Prefetch distances back to CPU
    cudaMemPrefetchAsync(d_distances, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_zero_counts, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_zero_indices, num_references * MAX_HITS * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize();

    // error checking
    size_t err_count = 0;
    for (int i = 0; i < num_references; i++) {
        int expected_zero_indices[MAX_HITS] = {0};
        int expected_zero_count = 0;
        int expected_lowest = 0;
        int expected_distance = bit_vector_levenshtein(query_length, reference_seqs[i], reference_lengths[i],
                                                       h_Eq, expected_zero_indices, &expected_zero_count, &expected_lowest);


        if (d_distances[i] != expected_distance || d_zero_counts[i] != expected_zero_count || d_lowest[i] != expected_lowest) {
            err_count++;
            printf("Mismatch in result for reference %d:\n", i);
            printf(" - Expected distance: %d, CUDA distance: %d\n", expected_distance, d_distances[i]);
            printf(" - Expected zero count: %d, CUDA zero count: %d\n", expected_zero_count, d_zero_counts[i]);
            printf(" - Expected lowest: %d, CUDA lowest: %d\n", expected_lowest, d_lowest[i]);


            printf(" - Expected zero indices: ");
            for (int j = 0; j < expected_zero_count; j++) {
                printf("%d ", expected_zero_indices[j]);
            }
            printf("\n - CUDA zero indices: ");
            for (int j = 0; j < d_zero_counts[i]; j++) {
                printf("%d ", d_zero_indices[i * MAX_HITS + j]);
            }
            printf("\n");
        }
    }

    printf("Query Sequence:\n%s\n\n", query);

    printf("Reference Sequences:\n");
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }


    // Print results
    for (int i = 0; i < num_references; i++) {
        printf("Results for Reference #%d:\n", i + 1);
        printf("Distance: %d, Lowest: %d, Hits (zero score positions): %d\n",
              d_distances[i], d_lowest[i], d_zero_counts[i]);
        printf("Zero Indices: ");
        for (int k = 0; k < d_zero_counts[i]; k++) {
            printf("%d ", d_zero_indices[i * MAX_HITS + k]);
        }
        printf("\n\n");
    }

    printf("Error count (CUDA program): %zu\n\n", err_count);

    // Free memory
    cudaFree(references);
    cudaFree(d_reference_lengths);
    cudaFree(d_distances);
    cudaFree(d_zero_counts);
    cudaFree(d_zero_indices);
    cudaFree(d_lowest);

    for (int i = 0; i < num_references; i++) {
      free((void*)reference_seqs[i]);
    }
    free(reference_seqs);
    free(reference_lengths);

    return 0;
}


Overwriting CUDA.cu


In [15]:
%%shell
nvcc -o CUDA CUDA.cu C_utils.c -arch=sm_75

In [16]:
%%shell
nvprof ./CUDA

==6541== NVPROF is profiling process 6541, command: ./CUDA
*** function = Levenshtein Distance
numReferences = 87
numBlocks = 1, numThreads = 256
Query Sequence:
TCC

Reference Sequences:
Reference #1:
TAGCAACGGTAGCAGTCGTAAAGGTGGTAGTAGAATTTATATTAGTACTATCAGCACATCAAAATAGTGTTTGCTTTGGAGAGAAGAAATCACGGAATACCGGCCAATCCCTCAACTACGAGATACCCCGTCTGAGCGCACAGGCATTGTTCTGTGGCGCGTAGGAATGCCCTTCCCTGCATCCGCTAGCGATAGATTGCATGCTCAGCTGCCGTCTGTGCGGGATCGCTTGGCCTCACGCTCAGCCTCGCCTCCCGTCGGCGGGCGCCTGACTGCAGCCACTGCTCGCAGTGTGTCAGCCCGCGTGAACGCGTGCATCAGCACAGCGCGTGCGTTGGTGCCGAGAGCGCGCTTTGACAGATCGCCATTGAGCTGTAGTGCCTCGCCGAGATTAGCAATGTCGTAGGGCGTCCGCAAGTCCAATGCCTGGTGCGCGCCGCTTGTCCACACTAGTCCACGGCCCCGTGTCGCGCGCACAACGCCTGCTGCATTGCTAACCCATTGCTGTCGCGTGGTCGCGTCTACTAGCGCGGGGCGATATGCGATCTCTAGCGATAGCCCGTTTGCTAGCGCCTGGCCAACCATCTTGTGCTTTGCAAAAAATCCCCACCGTGCCCCCATATCCAGCGTTACAAGATCAACCGCATCCCAGGCGCTGCTGCACGCTGCAAGAAGTATCTTTTCTGACACTGGCTGCACCGCGACAACGTCGTACTCGCTCGCTGTTGGGCTGCTGTTGCTGAGGCTATGCCCATGCGCTGCATCTGAAATCACCGCTGTCACCCTGCGCAGTATGCG

## CUDA - Multiple Query & Multiple Reference

In [17]:
%%writefile CUDA_Multiple.cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include "C_utils.h"

// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)
#define MAX_HITS 1024

// params to change here ↓↓↓

#define query_file "Query_3.fasta"
#define reference_file "Reference_87.fasta"
#define threadsPerBlock 256
#define loope 10

// params to change here ↑↑↑


typedef uint64_t bitvector;
__constant__ bitvector d_Eq[256];

__host__ __device__ int bit_vector_levenshtein(int query_length, const char *reference, int reference_length, const bitvector *Eq, int *zero_indices, int *zero_count, int* lowest) {
    if (query_length > MAX_LENGTH || reference_length > MAX_LENGTH) {
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Ph = 0;
    bitvector Mh = 0;
    bitvector Xv = 0;
    bitvector Xh = 0;
    bitvector Xp = 0;
    int score = query_length;


    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; j++) {
        unsigned char c = reference[j];

        Xv = Eq[c] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((((Xv & Pv) + Pv) ^ Pv) | Xv | Mv);
        Ph = Mv | ~(Xh | Pv);

        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (query_length - 1))) score++;
        if (Mh & (1ULL << (query_length - 1))) score--;
        if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }
    return score;
}

__global__ void levenshtein_kernel(int num_queries, int num_references, const char *__restrict__ d_queries, const int *__restrict__ d_query_lengths, const bitvector *__restrict__ d_Eq_queries,
                                   const char *__restrict__ d_references, const int *__restrict__ d_reference_lengths, int *__restrict__ d_distances, int *__restrict__ d_zero_counts,
    int *__restrict__ d_zero_indices,
    int *__restrict__ d_lowest)
{
    int query_idx = blockIdx.x;
    int reference_idx = blockIdx.y * blockDim.x + threadIdx.x;

    if (query_idx < num_queries && reference_idx < num_references) {
        int query_length = d_query_lengths[query_idx];  // Now matches parameter name

        const char *reference = &d_references[reference_idx * MAX_LENGTH];
        int reference_length = d_reference_lengths[reference_idx];

        const bitvector *Eq = &d_Eq_queries[query_idx * 256];

        int *current_zero_indices = &d_zero_indices[(query_idx * num_references + reference_idx) * MAX_HITS];
        int *current_zero_count = &d_zero_counts[query_idx * num_references + reference_idx];

        d_distances[query_idx * num_references + reference_idx] =
            bit_vector_levenshtein(query_length, reference, reference_length,
                                 Eq, current_zero_indices, current_zero_count,
                                 &d_lowest[query_idx * num_references + reference_idx]);
    }
}

int main(){

    // Read Query
    int num_queries = 0;
    char **query_seqs = parse_fasta_file(query_file, &num_queries);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    if (!reference_seqs) {
        for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
        free(query_seqs);
        return -1;
    }

    int *reference_lengths = (int*)malloc(num_references * sizeof(int));
    for (int i = 0; i < num_references; i++) {
        reference_lengths[i] = strlen(reference_seqs[i]);
    }

    // Precompute Eq for the query and store it in host memory.
    bitvector *h_Eq_queries = (bitvector*)malloc(num_queries * 256 * sizeof(bitvector));
    int *query_lengths = (int*)malloc(num_queries * sizeof(int));
    char **queries_input = (char**)malloc(num_queries * sizeof(char*));

    for (int q = 0; q < num_queries; q++) {
        queries_input[q] = query_seqs[q];
        query_lengths[q] = strlen(queries_input[q]);
        bitvector *h_Eq = &h_Eq_queries[q * 256];
        memset(h_Eq, 0, 256 * sizeof(bitvector));

        for (int i = 0; i < query_lengths[q]; i++) {
            h_Eq[(unsigned char)query_seqs[q][i]] |= (1ULL << i);
        }
    }

    bitvector *d_Eq_queries;
    cudaMalloc(&d_Eq_queries, num_queries * 256 * sizeof(bitvector));
    cudaMemcpy(d_Eq_queries, h_Eq_queries, num_queries * 256 * sizeof(bitvector), cudaMemcpyHostToDevice);

    // Allocate Unified Memory
    char *d_queries, *references;
    int *d_query_lengths, *d_reference_lengths, *d_distances, *d_zero_indices, *d_zero_counts, *d_lowest;

    cudaMallocManaged(&d_queries, num_queries * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_query_lengths, num_queries * sizeof(int));
    cudaMallocManaged(&d_Eq_queries, num_queries * 256 * sizeof(bitvector));

    cudaMallocManaged(&references, num_references * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_reference_lengths, num_references * sizeof(int));
    cudaMallocManaged(&d_distances, num_queries * num_references * sizeof(int)); //
    cudaMallocManaged(&d_zero_counts, num_queries * num_references * sizeof(int)); //
    cudaMallocManaged(&d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int)); //
    cudaMallocManaged(&d_lowest, num_references * sizeof(int));

    // copy ref and ref_length to managed memory
    for (int i = 0; i < num_references; i++) {
        strncpy(&references[i * MAX_LENGTH], reference_seqs[i], MAX_LENGTH);
        d_reference_lengths[i] = reference_lengths[i];
    }

    for (int q = 0; q < num_queries; q++) {
        strcpy(&d_queries[q * MAX_LENGTH], query_seqs[q]);
        d_query_lengths[q] = strlen(query_seqs[q]);
    }

    cudaMemcpy(d_Eq_queries, h_Eq_queries, num_queries * 256 * sizeof(bitvector), cudaMemcpyHostToDevice);

    // get gpu id
    int device = -1;
    cudaGetDevice(&device);

    // memory advise
    cudaMemAdvise(d_queries, num_queries * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_query_lengths, num_queries * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_Eq_queries, num_queries * 256 * sizeof(bitvector), cudaMemAdviseSetReadMostly, device);

    cudaMemAdvise(references, num_references * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_reference_lengths, num_references * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_distances, num_queries * num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device); //
    cudaMemAdvise(d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int), cudaMemAdviseSetPreferredLocation, device); //
    cudaMemAdvise(d_zero_counts, num_queries * num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device); //
    cudaMemAdvise(d_lowest, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);

    // prefetch data
    cudaMemPrefetchAsync(d_queries, num_queries * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_query_lengths, num_queries * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_Eq_queries, num_queries * 256 * sizeof(bitvector), device, NULL);

    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_distances, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_counts, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), device, NULL);

    // setup CUDA kernel
    int blocksPerGrid = (num_references + threadsPerBlock - 1) / threadsPerBlock;

    // print infos
    printf("*** function = Levenshtein Distance\n");
    printf("numQueries = %d\n", num_queries);
    printf("numReferences = %d\n", num_references);
    printf("numBlocks = %d, numThreads = %d\n", blocksPerGrid, threadsPerBlock);

    for (int i = 0; i < num_queries; i++) {
        printf("Query #%d:\n%s\n\n", i + 1, query_seqs[i]);
    }
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }

    // Execute kernel multiple times
    dim3 gridDim(num_queries); // One block per query
    dim3 blockDim(min(num_references, 1024));
    for (size_t i = 0; i < loope; i++) {
        levenshtein_kernel<<<gridDim, blockDim>>>(
        num_queries, num_references,
        d_queries, d_query_lengths, d_Eq_queries,
        references, d_reference_lengths,
        d_distances, d_zero_counts, d_zero_indices, d_lowest);
    }

    // barrier
    cudaDeviceSynchronize();

    // Prefetch distances back to CPU
    cudaMemPrefetchAsync(d_queries, num_queries * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_query_lengths, num_queries * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_Eq_queries, num_queries * 256 * sizeof(bitvector), device, NULL);

    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_distances, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_counts, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize();

    size_t err_count = 0;
    for (int q = 0; q < num_queries; q++) {
        printf("\nResults for Query #%d: \"%s\" (Length: %d)\n", q + 1, query_seqs[q], query_lengths[q]);

        for (int i = 0; i < num_references; i++) {
            int expected_zero_indices[MAX_HITS] = {0};
            int expected_zero_count = 0;
            int expected_lowest = 0;

            int expected_distance = bit_vector_levenshtein(query_lengths[q], reference_seqs[i], reference_lengths[i],
                                                          &h_Eq_queries[q * 256], expected_zero_indices, &expected_zero_count, &expected_lowest);

            if (d_distances[q * num_references + i] != expected_distance || d_zero_counts[q * num_references + i] != expected_zero_count || d_lowest[q * num_references + i] != expected_lowest) {
                err_count++;
                printf("Mismatch in result for reference %d:\n", i);
                printf(" - Expected distance: %d, CUDA distance: %d\n", expected_distance, d_distances[q * num_references + i]);
                printf(" - Expected zero count: %d, CUDA zero count: %d\n", expected_zero_count, d_zero_counts[q * num_references + i]);
                printf(" - Expected lowest: %d, CUDA lowest: %d\n", expected_lowest, d_lowest[q * num_references + i]);
                printf(" - Expected zero indices: ");
                for (int j = 0; j < expected_zero_count; j++) {
                    printf("%d ", expected_zero_indices[j]);
                }
                printf("\n - CUDA zero indices: ");
                for (int j = 0; j < d_zero_counts[q * num_references + i]; j++) {
                    printf("%d ", d_zero_indices[(q * num_references + i) * MAX_HITS + j]);
                }
                printf("\n");
            }
        }

        // Print results
        for (int i = 0; i < num_references; i++) {
            printf("Results for Query #%d & Reference #%d:\n",q+1,  i + 1);
            printf("Distance: %d, Lowest: %d, Hits (zero score positions): %d\n",
                  d_distances[q * num_references + i], d_lowest[q * num_references + i], d_zero_counts[q * num_references + i]);
            printf("Zero Indices: ");
            for (int k = 0; k < d_zero_counts[q * num_references + i]; k++) {
                printf("%d ", d_zero_indices[(q * num_references + i) * MAX_HITS + k]);
            }
            printf("\n\n");
        }
    }

    printf("Error count (CUDA program): %zu\n\n", err_count);

    // Free memory
    cudaFree(references);
    cudaFree(d_reference_lengths);
    cudaFree(d_distances);
    cudaFree(d_zero_counts);
    cudaFree(d_zero_indices);
    cudaFree(d_queries);
    cudaFree(d_query_lengths);
    cudaFree(d_Eq_queries);
    cudaFree(d_lowest);

    for (int i = 0; i < num_queries; i++) {
        free(query_seqs[i]);
    }
    free(query_seqs);
    free(query_lengths);

    for (int i = 0; i < num_references; i++) {
        free(reference_seqs[i]);
    }
    free(reference_seqs);
    free(reference_lengths);

    return 0;
}

Overwriting CUDA_Multiple.cu


In [18]:
%%shell
nvcc -o CUDA CUDA_Multiple.cu C_utils.c -arch=sm_75

In [19]:
%%shell
nvprof ./CUDA

==6676== NVPROF is profiling process 6676, command: ./CUDA
*** function = Levenshtein Distance
numQueries = 3
numReferences = 87
numBlocks = 1, numThreads = 256
Query #1:
TAG

Query #2:
TGGGTT

Query #3:
AGCGTT

Reference #1:
TAGCAACGGTAGCAGTCGTAAAGGTGGTAGTAGAATTTATATTAGTACTATCAGCACATCAAAATAGTGTTTGCTTTGGAGAGAAGAAATCACGGAATACCGGCCAATCCCTCAACTACGAGATACCCCGTCTGAGCGCACAGGCATTGTTCTGTGGCGCGTAGGAATGCCCTTCCCTGCATCCGCTAGCGATAGATTGCATGCTCAGCTGCCGTCTGTGCGGGATCGCTTGGCCTCACGCTCAGCCTCGCCTCCCGTCGGCGGGCGCCTGACTGCAGCCACTGCTCGCAGTGTGTCAGCCCGCGTGAACGCGTGCATCAGCACAGCGCGTGCGTTGGTGCCGAGAGCGCGCTTTGACAGATCGCCATTGAGCTGTAGTGCCTCGCCGAGATTAGCAATGTCGTAGGGCGTCCGCAAGTCCAATGCCTGGTGCGCGCCGCTTGTCCACACTAGTCCACGGCCCCGTGTCGCGCGCACAACGCCTGCTGCATTGCTAACCCATTGCTGTCGCGTGGTCGCGTCTACTAGCGCGGGGCGATATGCGATCTCTAGCGATAGCCCGTTTGCTAGCGCCTGGCCAACCATCTTGTGCTTTGCAAAAAATCCCCACCGTGCCCCCATATCCAGCGTTACAAGATCAACCGCATCCCAGGCGCTGCTGCACGCTGCAAGAAGTATCTTTTCTGACACTGGCTGCACCGCGACAACGTCGTACTCGCTCGCTGTTGGGCTGCTGTTGCTGAGGCTATGCCCATGCGCTGCATCTGAAATCAC